In [1]:
import sys

In [2]:
import os

In [3]:
from pathlib import Path

In [4]:
project_dir = Path(__name__).resolve().parents[1]

In [5]:
sys.path.insert(0, str(project_dir))

In [6]:
sys.path.insert(0, str(project_dir/'src'/'data'))

In [7]:
from scrape_bol import *

In [8]:
from searchterm import *

In [9]:
from datetime import datetime

In [27]:
s = SearchTerm(search_term='illy', search_domain = 'bol.com', max_pages = 1)

In [28]:
s.productlist

[Product(provider='bol', provider_id='9200000107510716', name='illy Classic Roast Lungo Koffiecups - 10 x 10 stuks - 100 koffiecups', url='https://bol.com/nl/nl/p/illy-classic-roast-lungo-koffiecups-10-x-10-stuks-100-koffiecups/9200000107510716/', price='32.93', original_price='43,90', hidden_price='32.93'),
 Product(provider='bol', provider_id='9300000003898934', name='illy Espresso Decaffeinato Koffiecups - 10 x 10 stuks - 100 koffiecups', url='https://bol.com/nl/nl/p/illy-espresso-decaffeinato-koffiecups-10-x-10-stuks-100-koffiecups/9300000003898934/', price='42.50', original_price=None, hidden_price='42.50'),
 Product(provider='bol', provider_id='9200000107510712', name='illy Espresso Intenso Koffiecups - 10 x 10 stuks - 100 koffiecups', url='https://bol.com/nl/nl/p/illy-espresso-intenso-koffiecups-10-x-10-stuks-100-koffiecups/9200000107510712/', price='42.50', original_price=None, hidden_price='42.50'),
 Product(provider='bol', provider_id='9200000072442692', name='illy Francis Fr

In [ ]:
len(s.productlist)

In [ ]:
print(s.Scraper.page)

In [ ]:
dir(s)

In [ ]:
datetime.now().strftime("%Y-%m-%d %I:%M%p")

In [ ]:
to_save_data = ['\t'.join([p.provider if p.provider else '', 
                          p.provider_id if p.provider_id else '', 
                           p.name  if p.name else '', 
                           p.price  if p.price else '', 
                           p.original_price  if p.original_price else '' ]) 
                for p in s.productlist]

In [ ]:
with open("../data/raw/price_tracker.tsv", 'a') as file:
    file.writelines(to_save_data)

In [10]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita')

In [11]:
pl_bs4 = bol.page.find_all('li', attrs = {'class': 'product-item--row'})

In [12]:
example = [p for p in pl_bs4 if p.attrs['data-id'] == '9200000074016762'][0]

In [237]:
example.find('span', attrs = {'data-test': 'price'}).contents[0].strip()

'42'

In [238]:
example.find('sup', attrs = {'data-test': 'price-fraction'}).contents[0].strip()

'94'

In [14]:
pl = bol.get_productlist()

In [15]:
pl

[]

In [16]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?page=8&searchtext=philips+sonicare&view=list') # No products

In [17]:
pl = bol.get_productlist()

In [21]:
bol._productlist_bs4

[]

In [22]:
bol.locators

{'product_search_result_locator': ('li',
  {'attrs': {'class': 'product-item--row'}}),
 'provider_id_locator': None,
 'name_locator': None,
 'url_locator': None,
 'price_locator': None,
 'original_price_locator': None}

In [18]:
pl

[]

In [ ]:
# pl

In [ ]:
from pathlib import Path

In [ ]:
project_dir = Path(__name__).resolve().parents[2]

In [ ]:
project_dir/'data'/'raw'/'example.csv'

# Create DB

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
conn = sqlite3.connect("../data/raw/price_tracker.db",  timeout=10)

In [ ]:
sql_create_products_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_prices_table = """CREATE TABLE IF NOT EXISTS prices (
                                datetime datetime NOT NULL,
                                provider text ,
                                provider_id text,
                                name text,
                                price numeric,
                                original_price numeric
                            );"""

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
create_table(conn, sql_create_prices_table)

In [ ]:
conn.close()

# Pandas

In [10]:
import pandas as pd

data_dir = project_dir/'data'/'external'
max_file = data_dir /'new_price.tsv'
second_max_file = data_dir / 'old_price.tsv'

df_max = pd.read_csv(max_file, sep = '\t')
df_second_max = pd.read_csv(second_max_file, sep = '\t')

In [11]:
for df in [df_max, df_second_max]:
    df['provider_id'] = df['provider_id'].astype('str')

df = pd.merge(df_second_max, df_max, on = ['provider', 'provider_id'], 
        suffixes = (None, '_old'))

df['discount'] = 1 - df['price'] / df['price_old']

df_notify = df[df['discount'] >=0.05]

In [13]:
df_notify = df[df['discount'] != 0]

In [17]:
df[df['name'].str.contains('Little Dutch Memorybox')]

,provider,provider_id,name,price,original_price,hidden_price,URL,name_old,price_old,original_price_old,hidden_price_old,URL_old,discount
77,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,23.69,https://bol.com/nl/nl/p/little-dutch-memorybox...,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,23.69,https://bol.com/nl/nl/p/little-dutch-memorybox...,0.0


In [23]:
df_notify['URL'].values[-1]

'https://bol.com/nl/nl/p/illy-iperespresso-koffie-home-cafeinevrij-6-x-18-capsules/9300000007109869/'

In [262]:
record = records[0]

In [263]:
records = df_notify.to_dict(orient = 'records')
r = []
for record in records:
    price = record['price']
    original_price = record['price_old']
    url = record['URL']

    r.append('\n'.join([record['provider'], record['name'], f'{price}~{original_price}~',
                            f'<{url}|Product Link>'
                           ])
            )

In [264]:
'\n\n'.join(r)

''

# Dotenv

In [ ]:
from dotenv import find_dotenv, load_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
f = project_dir.parent / 'Utils'

In [ ]:
f

# Selenium Driver

In [14]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita', driver_method = 'selenium')

In [15]:
pl = bol.get_productlist()

In [20]:
bol.driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="366ffb8fb0c52d2d66a149f2cd27670c")>

In [17]:
# pl

In [18]:
bol.close()

# MediaMarkt

In [11]:
from scrape_mediamarkt import *

In [12]:
url =  'https://www.mediamarkt.nl/nl/search.html?query=galaxy+tab+s7&searchProfile=onlineshop&channel=mmnlnl'

In [13]:
s = MediaMarktScraper(url, driver_method = 'selenium')

In [14]:
product_lists = s.page.find_all('div', attrs = {'class':'product-wrapper'})

In [15]:
s.productlist

[Product(provider='mediamarkt', provider_id='MMNL1687668', name='SAMSUNG Galaxy Tab S7+ 256 GB WIFI Blauw', url='https://mediamarkt.nl/nl/product/_samsung-galaxy-tab-s7-256-gb-wifi-blauw-1687668.html', price='849.', original_price='979.', hidden_price=None),
 Product(provider='mediamarkt', provider_id='MMNL1697849', name='SAMSUNG GALAXY TAB S7 128GB LTE - Enterprise Edition - BLACK', url='https://mediamarkt.nl/nl/product/_samsung-galaxy-tab-s7-128gb-lte-enterprise-edition-black-1697849.html', price='669.', original_price='669.', hidden_price=None),
 Product(provider='mediamarkt', provider_id='MMNL1687667', name='SAMSUNG Galaxy Tab S7+ 128 GB WIFI Blauw', url='https://mediamarkt.nl/nl/product/_samsung-galaxy-tab-s7-128-gb-wifi-blauw-1687667.html', price='841.', original_price='899.', hidden_price=None),
 Product(provider='mediamarkt', provider_id='MMNL1687665', name='SAMSUNG Galaxy Tab S7 128 GB WIFI Blauw', url='https://mediamarkt.nl/nl/product/_samsung-galaxy-tab-s7-128-gb-wifi-blauw-

In [106]:
product = product_lists[0]

In [126]:
prices = product.find('div', attrs = {'class': 'price', 'class': 'small'})

In [127]:
''.join([i.contents[0] for i in prices.contents]).strip().strip('-').replace(',', '.')

'849.'

In [128]:
old_prices = product.find('div', attrs = {'class': 'price-old', 'class': 'price'})

In [129]:
''.join([i.contents[0] for i in old_prices.contents]).strip().strip('-').replace(',', '.')

'979.'

In [65]:
'https://mediamarkt.nl'+ product.find('a').attrs['data-clickable-href']

'https://mediamarkt.nl/nl/product/_philips-sonicare-plaquedefense-hx6231-01-1371881.html'

In [66]:
product.attrs

{'class': ['product-wrapper'],
 'data-bundle': 'false',
 'data-modelnumber': '1371881',
 'data-age-rating': '0',
 'data-gtm-impression-event': 'EEC_PRODUCT_IMPRESSION',
 'data-gtm-impression-event-ext': 'product1371881',
 'data-reco-pid': 'MMNL1371881'}

In [79]:
product.find('img').attrs['alt']

'PHILIPS Sonicare PlaqueDefense HX6231/01'

In [16]:
s.close()